# Evaluating an MT5 model for Czech paraphrasing

In [1]:
model = 'yawnick/mt5-small-paracrawl-cscs' 
dataset = 'yawnick/para_crawl_cscs'

## Environment Setup

We need a GPU

In [2]:
!nvidia-smi

Wed May 24 07:02:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

We install all needed libraries

In [3]:
!pip install datasets==2.11.0 transformers==4.28.0 nltk==3.8.1 parascore==1.0.5 sentencepiece==0.1.98

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

Connect to Google Drive to save the results in the root folder of our Drive at `/content/drive/MyDrive/`.

In [4]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


## Data Download and Preparation

In [5]:
from datasets import load_dataset

We use our own created datasets with paraphrases

In [6]:
raw_dataset = load_dataset(dataset, split='test')
raw_dataset[5]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/yawnick___csv/yawnick--para_crawl_cscs-8346f3eb1e22ef37/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


{'Original': 'c) stanoví odpovídající pokuty nebo jiné sankce k úèinnému zabezpeèení plnìní tohoto èlánku.',
 'Paraphrase': 'c) Pro zajištění účinného výkonu tohoto článku stanoví vhodné sankce nebo jiné sankce.'}

## Generating paraphrases

First, we will initialize the pipeline

In [7]:
from transformers import pipeline
import tensorflow as tf
from tqdm import tqdm

In [8]:
device_name = tf.test.gpu_device_name()
if len(device_name) > 0:
    print("Found GPU at: {}".format(device_name))
else:
    device_name = "/device:CPU:0"
    print("No GPU, using {}.".format(device_name))

Found GPU at: /device:GPU:0


In [9]:
pipe = pipeline('text2text-generation', model=model)  # device=0 tells it to use the GPU

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
def data():
  for row in raw_dataset:
    yield row['Original']

In [11]:
ds_length = raw_dataset.num_rows
ps = []

with tf.device(device_name):
  for res in tqdm(pipe(data(), batch_size=48), total=ds_length):
    ps.append(res[0]['generated_text'])

100%|██████████| 11532/11532 [39:38<00:00,  4.85it/s]


## Evaluating paraphrases

In [12]:
from parascore import ParaScorer

In [13]:
scorer = ParaScorer(lang='cs')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
scores = scorer.base_score(raw_dataset['Original'], ps, raw_dataset['Paraphrase'], batch_size=16)
scores

[0.9497106166192943,
 0.9500001192092895,
 0.9388387966156005,
 0.9499998211860656,
 0.95,
 0.8315376210212707,
 0.9671427379335676,
 0.913268404006958,
 0.95,
 0.9386431809979007,
 0.9363115000724792,
 0.8996621720512192,
 0.9789359140396118,
 0.8697255073987764,
 0.9333073757716588,
 0.95,
 0.9158138939312526,
 0.9149269688835543,
 0.8464347290992736,
 0.9252170559028526,
 0.9572608637809753,
 0.95,
 0.9132377117200006,
 0.8703486490249633,
 0.9355955505571445,
 0.9197050499916076,
 0.9375852513313293,
 0.9408191253612568,
 0.9276096272869271,
 0.884933488368988,
 0.8604777215894606,
 0.9596889781951904,
 0.9499999403953552,
 0.9499998807907104,
 0.9196972795895167,
 0.9899015970418019,
 1.0153753026634382,
 0.9682192373275756,
 0.8884111874444144,
 0.9499998807907104,
 0.9499999403953552,
 0.95,
 0.9500001192092895,
 0.9156016823533294,
 0.8853435550417219,
 0.8572591114044189,
 0.7618666478565761,
 0.841547949329368,
 0.9399671156603591,
 0.8520491886138916,
 0.887125985622406,
 0.

Print the average Parascore

In [15]:
score = sum(scores) / len(scores)
print('Average Parascore:', score)

Average Parascore: 0.9217165390213743


Generate and export the evaluation table

In [16]:
raw_dataset = raw_dataset.rename_column('Paraphrase', 'Reference')
raw_dataset = raw_dataset.add_column(name='Paraphrase', column=ps)
raw_dataset = raw_dataset.add_column(name='Parascore', column=scores)
raw_dataset = raw_dataset.to_csv('/content/drive/MyDrive/data/eval_table_mono_cscs.csv')
raw_dataset

Creating CSV from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

3016998